In [3]:
'''
@author: Rohit Sharma
Date: 01-01-2023
Decription: Transformation of Olympics Data GCS
'''
import json
from google.cloud import storage
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("Olympics Analytics") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/01 13:36:43 INFO SparkEnv: Registering MapOutputTracker
24/01/01 13:36:44 INFO SparkEnv: Registering BlockManagerMaster
24/01/01 13:36:44 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/01/01 13:36:44 INFO SparkEnv: Registering OutputCommitCoordinator


In [4]:
spark

**Input Files**

In [7]:
# Define the GCS bucket and file path
gcs_bucket = "us-east1-test-env-olympic-ab93b90d-bucket"

athletes = "/dags/data/raw/Athletes.csv"
coaches = "/dags/data/raw/Coaches.csv"
entries_gender = "/dags/data/raw/EntriesGender.csv"
medals = "/dags/data/raw/Medals.csv"
teams = "/dags/data/raw/Teams.csv"

# Construct the full GCS URI
athletes = f"gs://{gcs_bucket}/{athletes}"
coaches = f"gs://{gcs_bucket}/{coaches}"
entries_gender = f"gs://{gcs_bucket}/{entries_gender}"
medals = f"gs://{gcs_bucket}/{medals}"
teams = f"gs://{gcs_bucket}/{teams}"

In [15]:
# Read data from GCS into a Spark DataFrame
df_athletes = spark.read.csv(athletes, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df_athletes.show()

+--------------------+--------------------+-------------------+
|                Name|                 NOC|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [16]:
df_athletes.printSchema()

root
 |-- Name: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [17]:
# Read data from GCS into a Spark DataFrame
df_coaches = spark.read.csv(coaches, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df_coaches.show()

+--------------------+--------------------+-----------------+--------+
|                Name|                 NOC|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|       Côte d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    null|
|     

In [18]:
df_coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [19]:
# Read data from GCS into a Spark DataFrame
df_entries_gender = spark.read.csv(entries_gender, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df_entries_gender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [27]:
df_entries_gender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [21]:
# Read data from GCS into a Spark DataFrame
df_medals = spark.read.csv(medals, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df_medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|            Team/NOC|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [22]:
df_medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team/NOC: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [23]:
# Read data from GCS into a Spark DataFrame
df_teams = spark.read.csv(teams, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df_teams.show()

+-------------+--------------+--------------------+------------+
|         Name|    Discipline|                 NOC|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [24]:
df_teams.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Event: string (nullable = true)



In [30]:
# Find the top countries with the highest number of gold medals
top_gold_medal_countries = df_medals.orderBy("Gold", ascending=False).select("Team/NOC","Gold").show()

+--------------------+----+
|            Team/NOC|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|               Italy|  10|
|             Germany|  10|
|              France|  10|
|              Canada|   7|
|         New Zealand|   7|
|              Brazil|   7|
|                Cuba|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|              Poland|   4|
|      Czech Republic|   4|
|               Kenya|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [31]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = df_entries_gender.withColumn(
    'Avg_Female', df_entries_gender['Female'] / df_entries_gender['Total']
).withColumn(
    'Avg_Male', df_entries_gender['Male'] / df_entries_gender['Total']
)
average_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

**Output Files**

In [36]:
output_path_athletes = "/dags/data/trusted/athletes"
output_path_coaches = "/dags/data/trusted/coaches"
output_path_entriesgender = "/dags/data/trusted/entriesgender"
output_path_medals = "/dags/data/trusted/medals"
output_path_teams = "/dags/data/trusted/teams"

# Construct the full GCS URI
gcs_output_athletes = f"gs://{gcs_bucket}/{output_path_athletes}"
gcs_output_coaches = f"gs://{gcs_bucket}/{output_path_coaches}"
gcs_output_entriesgender = f"gs://{gcs_bucket}/{output_path_entriesgender}"
gcs_output_medals = f"gs://{gcs_bucket}/{output_path_medals}"
gcs_output_teams = f"gs://{gcs_bucket}/{output_path_teams}"

# Write the DataFrame to GCS
df_athletes.repartition(1).write.mode("overwrite").csv(gcs_output_athletes, header=True)
df_coaches.repartition(1).write.mode("overwrite").csv(gcs_output_coaches, header=True)
average_entries_by_gender.repartition(1).write.mode("overwrite").csv(gcs_output_entriesgender, header=True)
df_medals.repartition(1).write.mode("overwrite").csv(gcs_output_medals, header=True)
df_teams.repartition(1).write.mode("overwrite").csv(gcs_output_teams, header=True)